In [16]:
import numpy as np
import pandas
from sklearn.preprocessing import Normalizer
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.optimizers import RMSprop

In [17]:
# Get data
heart_disease = fetch_ucirepo(id=45)

X = heart_disease.data.features
y = heart_disease.data.targets

In [18]:
# deleting rows with NAN
X['y']=y
X=X.dropna(how='any')
y=X['y']
X=X.drop('y', axis=1)

# replace disease stadium with one information- sick
y = y.replace([2, 3, 4], 1)

# Transforming categorical values into binary, one-hot encoded values
X = pandas.get_dummies(X, columns=['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal'], dtype=float)

# normalize data
normalizer = Normalizer().fit(X)
X_nm = normalizer.transform(X)
X_nm = pandas.DataFrame(X_nm, columns=X.columns)

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X_nm, y, test_size=0.25, random_state=65)

# preparing torch tensor datasets
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.float)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.float)

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)

In [20]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_layer_sizes):
        super(NeuralNetwork, self).__init__()
        self.hidden_layers = nn.ModuleList()
        prev_layer_output_size = input_size 

        # creating hidden layers
        for layer_size in hidden_layer_sizes:
            hidden_layer = nn.Linear(prev_layer_output_size, layer_size)
            self.hidden_layers.append(hidden_layer)
            prev_layer_output_size = layer_size
        print(self.hidden_layers)
            
        # output layer
        self.output_layer = nn.Linear(prev_layer_output_size, 1)
        
        # activation function for hidden layers
        self.activation_function = nn.ReLU()

    def forward(self, x):
        # coming through hiddden layers
        for hidden_layer in self.hidden_layers:
            x = self.activation_function(hidden_layer(x))
        
        # output calculated with sigmoid
        x = torch.sigmoid(self.output_layer(x))
        return x

In [21]:
def train_model(model, criterion, optimizer, train_loader, epochs=2000):
    for epoch in range(epochs):
        model.train()
        for i, (inputs, targets) in enumerate(train_loader):
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [22]:
def evaluate_metrics(model, data_loader):
    with torch.no_grad():
        Y_true, Y_pred = [], []
        for features, targets in data_loader:
            outputs = model(features)
            predicted = (outputs.squeeze() > 0.5).float()
            Y_true.extend(targets.tolist())
            Y_pred.extend(predicted.tolist())
    print('Accuracy:', accuracy_score(Y_true, Y_pred), 'Precision:', precision_score(Y_true, Y_pred), 'F1-Score:', f1_score(Y_true, Y_pred))

In [23]:
input_size = X_train.shape[1]
hidden_layers = [16, 7]
model = NeuralNetwork(input_size, hidden_layers)

learning_rates = [0.0001, 0.001, 0.01, 0.1, 1]
batch_sizes = [3, 5, 10, 20] 
optimizers = [torch.optim.SGD, torch.optim.Adam, torch.optim.RMSprop]
O_names = ["SGD", "Adam", "RMSprop"]

ModuleList(
  (0): Linear(in_features=25, out_features=16, bias=True)
  (1): Linear(in_features=16, out_features=7, bias=True)
)


In [24]:
def training(lr, opt_name, optimizer, batch_size, epochs):
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
    
    # model = NeuralNetwork(input_size, hidden_layers)

    criterion = nn.BCELoss()
    print(opt_name, "Learning rate:", lr, "Batch:", batch_size)
    train_model(model, criterion, optimizer(model.parameters(), lr=lr), train_loader, epochs)
    evaluate_metrics(model, test_loader)

In [25]:
for lr in learning_rates:
    for opt_name, optimizer in zip(O_names, optimizers):
        for batch_size in batch_sizes:
            training(lr, opt_name, optimizer, batch_size, epochs)

SGD Learning rate: 0.0001 Batch: 3


c:\Users\505ry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.52 Precision: 0.0 F1-Score: 0.0
SGD Learning rate: 0.0001 Batch: 5


c:\Users\505ry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.52 Precision: 0.0 F1-Score: 0.0
SGD Learning rate: 0.0001 Batch: 10


c:\Users\505ry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.52 Precision: 0.0 F1-Score: 0.0
SGD Learning rate: 0.0001 Batch: 20


c:\Users\505ry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.52 Precision: 0.0 F1-Score: 0.0
Adam Learning rate: 0.0001 Batch: 3
Accuracy: 0.84 Precision: 0.9 F1-Score: 0.8181818181818182
Adam Learning rate: 0.0001 Batch: 5
Accuracy: 0.8266666666666667 Precision: 0.896551724137931 F1-Score: 0.7999999999999999
Adam Learning rate: 0.0001 Batch: 10
Accuracy: 0.8266666666666667 Precision: 0.896551724137931 F1-Score: 0.7999999999999999
Adam Learning rate: 0.0001 Batch: 20
Accuracy: 0.8266666666666667 Precision: 0.896551724137931 F1-Score: 0.7999999999999999
RMSprop Learning rate: 0.0001 Batch: 3
Accuracy: 0.8533333333333334 Precision: 0.9032258064516129 F1-Score: 0.835820895522388
RMSprop Learning rate: 0.0001 Batch: 5
Accuracy: 0.8533333333333334 Precision: 0.9032258064516129 F1-Score: 0.835820895522388
RMSprop Learning rate: 0.0001 Batch: 10
Accuracy: 0.8533333333333334 Precision: 0.9032258064516129 F1-Score: 0.835820895522388
RMSprop Learning rate: 0.0001 Batch: 20
Accuracy: 0.8533333333333334 Precision: 0.9032258064516129 F1-Score: 0.

KeyboardInterrupt: 